In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.client import device_lib
print(tf.__version__)
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()


1.2.1


[u'/gpu:0', u'/gpu:1']

# Task 3

In [2]:
train_data = pd.read_csv("../../data/heart_train.csv")
test_data = pd.read_csv("../../data/heart_test.csv")

print train_data.info()
print [name for name in train_data.columns]
print train_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 10 columns):
sbp          413 non-null int64
tobacco      413 non-null float64
ldl          413 non-null float64
adiposity    413 non-null float64
famhist      413 non-null object
typea        413 non-null int64
obesity      413 non-null float64
alcohol      413 non-null float64
age          413 non-null int64
chd          413 non-null int64
dtypes: float64(5), int64(4), object(1)
memory usage: 32.3+ KB
None
['sbp', 'tobacco', 'ldl', 'adiposity', 'famhist', 'typea', 'obesity', 'alcohol', 'age', 'chd']
   sbp  tobacco   ldl  adiposity  famhist  typea  obesity  alcohol  age  chd
0  160    12.00  5.73      23.11  Present     49    25.30    97.20   52    1
1  144     0.01  4.41      28.61   Absent     55    28.87     2.06   63    1
2  118     0.08  3.48      32.28  Present     52    29.14     3.81   46    0
3  170     7.50  6.41      38.03  Present     51    31.99    24.26   58    1
4  134    13.60 

# Logistic Regression on heart data

In [3]:
TRAIN_DATA_PATH = '../../data/heart_train.csv'
TEST_DATA_PATH = '../../data/heart_test.csv'

In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

print("=========== training ===============")
pd_train_data = pd.read_csv(TRAIN_DATA_PATH)
n_train_data, _ = pd_train_data.shape
print n_train_data

print pd_train_data.head()
pd_train_data["famhist"] = pd_train_data["famhist"].astype('category')
pd_train_data["famhist"] = pd_train_data["famhist"].cat.codes
train_labels = np.array(pd_train_data.chd)
pd_train_data.drop("chd", axis=1, inplace=True)
print pd_train_data.head()
train_data = np.float64(pd_train_data)
print train_data.shape

log_reg = LogisticRegression()
log_reg.fit(train_data, train_labels)



=========== training ===============
413
   sbp  tobacco   ldl  adiposity  famhist  typea  obesity  alcohol  age  chd
0  160    12.00  5.73      23.11  Present     49    25.30    97.20   52    1
1  144     0.01  4.41      28.61   Absent     55    28.87     2.06   63    1
2  118     0.08  3.48      32.28  Present     52    29.14     3.81   46    0
3  170     7.50  6.41      38.03  Present     51    31.99    24.26   58    1
4  134    13.60  3.50      27.78  Present     60    25.99    57.34   49    1
   sbp  tobacco   ldl  adiposity  famhist  typea  obesity  alcohol  age
0  160    12.00  5.73      23.11        1     49    25.30    97.20   52
1  144     0.01  4.41      28.61        0     55    28.87     2.06   63
2  118     0.08  3.48      32.28        1     52    29.14     3.81   46
3  170     7.50  6.41      38.03        1     51    31.99    24.26   58
4  134    13.60  3.50      27.78        1     60    25.99    57.34   49
(413, 9)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [5]:
print("=========== testing ===============")

pd_test_data = pd.read_csv(TEST_DATA_PATH)
n_test_data, _ = pd_test_data.shape
print n_test_data

print pd_test_data.head()
pd_test_data["famhist"] = pd_test_data["famhist"].astype('category')
pd_test_data["famhist"] = pd_test_data["famhist"].cat.codes
test_labels = np.array(pd_test_data.chd)
pd_test_data.drop("chd", axis=1, inplace=True)
print pd_test_data.head()
test_data = np.float64(pd_test_data)
print test_data.shape

test_prob = log_reg.predict(pd_test_data)
log_reg.predict(test_data)
log_reg.score(test_data, test_labels)

=========== testing ===============
49
   sbp  tobacco   ldl  adiposity  famhist  typea  obesity  alcohol  age  chd
0  138     6.00  7.24      37.05   Absent     38    28.69     0.00   59    0
1  148     0.00  5.32      26.71  Present     52    32.21    32.78   27    0
2  124     4.20  2.94      27.59   Absent     50    30.31    85.06   30    0
3  118     1.62  9.01      21.70   Absent     59    25.89    21.19   40    0
4  116     4.28  7.02      19.99  Present     68    23.31     0.00   52    1
   sbp  tobacco   ldl  adiposity  famhist  typea  obesity  alcohol  age
0  138     6.00  7.24      37.05        0     38    28.69     0.00   59
1  148     0.00  5.32      26.71        1     52    32.21    32.78   27
2  124     4.20  2.94      27.59        0     50    30.31    85.06   30
3  118     1.62  9.01      21.70        0     59    25.89    21.19   40
4  116     4.28  7.02      19.99        1     68    23.31     0.00   52
(49, 9)


0.77551020408163263

## using tensorflow

In [24]:
BATCH_SIZE = 3
N_FEATURES = 9
BETA = 0.1 # regularizer
EPOCHS = 1000
LEARNING_RATE=0.01

### data1 

In [25]:
# Step 1: get data

def batch_generator(filenames):
    """ filenames is the list of files you want to read from. 
    In this case, it contains only heart.csv
    """
    filename_queue = tf.train.string_input_producer(filenames)
    reader = tf.TextLineReader(skip_header_lines=1) # skip the first line in the file
    _, value = reader.read(filename_queue)

    # record_defaults are the default values in case some of our columns are empty
    # This is also to tell tensorflow the format of our data (the type of the decode result)
    # for this dataset, out of 9 feature columns, 
    # 8 of them are floats (some are integers, but to make our features homogenous, 
    # we consider them floats), and 1 is string (at position 5)
    # the last column corresponds to the lable is an integer

    record_defaults = [[1.0] for _ in range(N_FEATURES)]
    record_defaults[4] = ['']
    record_defaults.append([1])

    # read in the 10 rows of data
    content = tf.decode_csv(value, record_defaults=record_defaults) 

    # convert the 5th column (present/absent) to the binary value 0 and 1
    condition = tf.equal(content[4], tf.constant('Present'))
    content[4] = tf.cond(condition, lambda: tf.constant(1.0), lambda: tf.constant(0.0))

    # pack all 9 features into a tensor
    features = tf.stack(content[:N_FEATURES])
    # assign the last column to label
    labels = tf.stack([1 - content[-1], content[-1]], 0)

    # minimum number elements in the queue after a dequeue, used to ensure 
    # that the samples are sufficiently mixed
    # I think 10 times the BATCH_SIZE is sufficient
    min_after_dequeue = 10 * BATCH_SIZE

    # the maximum number of elements in the queue
    capacity = 20 * BATCH_SIZE

    # shuffle the data to generate BATCH_SIZE sample pairs
    data_batch, label_batch = tf.train.shuffle_batch([features, labels], batch_size=BATCH_SIZE, 
                                        capacity=capacity, min_after_dequeue=min_after_dequeue)

    return data_batch, label_batch

def generate_batches(data_batch, label_batch):
    with tf.Session() as sess:
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        for _ in range(10): # generate 10 batches
            features, labels = sess.run([data_batch, label_batch])
            print features, labels
        coord.request_stop()
        coord.join(threads)

data1_feature_batch, data1_label_batch = batch_generator([TRAIN_DATA_PATH])
test_data1_feature_batch, test_data1_label_batch = batch_generator([TEST_DATA_PATH])

# generate_batches(data1_feature_batch, data1_label_batch)
# generate_batches(test_data1_feature_batch, test_data1_label_batch)

### DEFINITIONS

In [32]:

# Step 2: create placeholders for input X (Features) and label Y (binary result)
X = tf.placeholder(tf.float32, shape=[None, 9], name="X")
Y = tf.placeholder(tf.float32, shape=[None,2], name="Y")

# Step 3: create weight and bias, initialized to 0
w = tf.Variable(tf.truncated_normal([9, 2]), name="weights")
b = tf.Variable(tf.zeros([1,2]), name="bias")

print("X",X)
print("Y",Y)
print("w",w)
print("b",b)

# Step 4: logistic multinomial regression / softmax
score = tf.matmul(X, w) + b

# Step 5: define loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=score, labels=Y, name="entropy")

regularizer = tf.nn.l2_loss(w)
loss = tf.reduce_sum(entropy + BETA * regularizer, name="loss")

# Step 6: using gradient descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE).minimize(loss)

print("score",score)
print("loss", loss)
print("optimizer", optimizer)

# Step 7: Prediction
Y_predicted = tf.nn.softmax(tf.matmul(X, w) + b)
correct_prediction = tf.equal(tf.argmax(Y_predicted,1), tf.argmax(Y,1))
accuracy = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))

print("Y_predicted", Y_predicted)
print("correct_prediction", correct_prediction)
print("accuracy", accuracy)

('X', <tf.Tensor 'X_13:0' shape=(?, 9) dtype=float32>)
('Y', <tf.Tensor 'Y_13:0' shape=(?, 2) dtype=float32>)
('w', <tf.Variable 'weights_13:0' shape=(9, 2) dtype=float32_ref>)
('b', <tf.Variable 'bias_13:0' shape=(1, 2) dtype=float32_ref>)
('score', <tf.Tensor 'add_19:0' shape=(?, 2) dtype=float32>)
('loss', <tf.Tensor 'loss_13:0' shape=() dtype=float32>)
('optimizer', <tf.Operation 'GradientDescent_13' type=NoOp>)
('Y_predicted', <tf.Tensor 'Softmax_13:0' shape=(?, 2) dtype=float32>)
('correct_prediction', <tf.Tensor 'Equal_19:0' shape=(?,) dtype=bool>)
('accuracy', <tf.Tensor 'Sum:0' shape=() dtype=float32>)


### This method did not workout
https://stackoverflow.com/questions/37091899/how-to-actually-read-csv-data-in-tensorflow

### TRAINING

In [30]:
import glob, os
for f in glob.glob("/tmp/model.ckpt*"):
    os.remove(f)

saver = tf.train.Saver([w,b])
EPOCHS = 1000
with tf.device("/gpu:1"):
    with tf.Session() as sess:
        # Step 7: initialize the necessary variables, in this case, w and b
        sess.run(tf.global_variables_initializer())

        # Step 8: train the model
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)

        n_batches = int(n_train_data/BATCH_SIZE)
        for epoch in tqdm(range(EPOCHS)): # run epochs
            avg_loss = 0

            for _ in range(n_batches):
                x_batch, y_batch = sess.run([data1_feature_batch, data1_label_batch])
                # Session runs train_op to minimize loss
                feed_dict={X: x_batch, Y:y_batch}
                _, loss_batch = sess.run([optimizer, loss], feed_dict=feed_dict)
                avg_loss += loss_batch/n_batches

            if (epoch+1) % 100 == 0:
                print "avg_loss",avg_loss

        coord.request_stop()
        coord.join(threads)

        # Step 9: saving the values of w and b
        print "weights",w.eval()
        print "bias",b.eval()

        # Add ops to save and restore all the variables.
        save_path = saver.save(sess, "/tmp/logit_reg_tf_model")



 10%|█         | 101/1000 [00:18<02:38,  5.66it/s]

avg_loss 52.3414892663


 20%|██        | 201/1000 [00:36<02:20,  5.70it/s]

avg_loss 61.8826450327


 30%|███       | 301/1000 [00:53<02:02,  5.69it/s]

...

avg_loss 56.3451955466


 90%|█████████ | 901/1000 [02:37<00:17,  5.67it/s]

avg_loss 59.6619875727


100%|██████████| 1000/1000 [02:55<00:00,  5.28it/s]


avg_loss 58.0269318993
weights [[ 0.8931703  -0.89317137]
 [-2.10434222  2.10434222]
 [-0.68888742  0.68888736]
 [-0.64374071  0.64374071]
 [-0.362014    0.362014  ]
 [-0.27913377  0.27913365]
 [ 1.54197383 -1.54197407]
 [ 0.16302009 -0.16302021]
 [-2.28227639  2.28227639]]
bias [[ 14.5101223  -14.51012039]]


### TESTING

In [35]:
# Step 10: predict
# test the model

saver = tf.train.import_meta_graph("/tmp/logit_reg_tf_model.meta")
with tf.device("/gpu:1"):
    with tf.Session() as sess:
        # nitialize the necessary variables, in this case, w and b
        sess.run(tf.global_variables_initializer())
        # Add ops to save and restore all the variables.
        saver.restore(sess, "/tmp/logit_reg_tf_model")
        print "weights",w.eval()
        print "bias",b.eval()

        total_correct_preds = 0
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)

        try:
            n_batches = int(n_test_data/BATCH_SIZE)
            for i in range(n_batches):
                x_batch, y_batch = sess.run([test_data1_feature_batch, test_data1_label_batch])
                total_correct_preds += sess.run(accuracy, feed_dict={X: x_batch, Y:y_batch})

        except tf.errors.OutOfRangeError:
            print('Done testing ...')
        coord.request_stop()
        coord.join(threads)

        print 'total_correct_preds {0}'.format(total_correct_preds)
        print 'n_test_data {0}'.format(n_test_data)
        print 'Accuracy {0}'.format(total_correct_preds/n_test_data)

INFO:tensorflow:Restoring parameters from /tmp/logit_reg_tf_model
weights [[ 0.82574016 -1.55367398]
 [ 0.69510132 -0.05141348]
 [-0.33732834  1.51485062]
 [ 0.89531285  0.15007007]
 [ 0.87078243 -0.48196244]
 [ 0.21861432  1.49471414]
 [-0.60588765  1.68290281]
 [ 0.83973187 -0.5009644 ]
 [ 1.66750026 -0.67938644]]
bias [[ 0.  0.]]
total_correct_preds 31.0
n_test_data 49
Accuracy 0.632653061224
